In [ ]:
from typing import Literal
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import MessagesState, StateGraph
from langgraph.prebuilt import ToolNode

from composio_langgraph import App, ComposioToolSet

In [2]:
load_dotenv()

True

In [3]:
# Initialize the toolset for GitHub
toolset = ComposioToolSet()
tools = toolset.get_tools(apps=[App.GOOGLECALENDAR])
tool_node = ToolNode(tools)

[2024-11-28 02:48:11,006][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this
/home/combi2k2/miniconda3/envs/AI-Agent/lib/python3.13/site-packages/composio/client/collections.py:1131: UserWarning: Using all the actions of an app is not recommended. Please use tags to filter actions or provide specific actions. We just pass the important actions to the agent, but this is not meant to be used in production. Check out https://docs.composio.dev/sdk/python/actions for more information.
  warnings.warn(


In [4]:
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=10,
    # other params...
)
model_with_tools = model.bind_tools(tools)

In [5]:
def call_model(state: MessagesState):
    messages = state["messages"]
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return "__end__"

In [6]:
workflow = StateGraph(MessagesState)

In [7]:
# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge("__start__", "agent")
workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")

In [8]:
app = workflow.compile()

In [9]:
SYSTEM_PROMPT = """
    You are an AI agent responsible for taking actions on Google Calendar on users' behalf. 
    You need to take action on Calendar using Google Calendar APIs. Use correct tools to run APIs from the given tool-set.
"""

In [11]:
from datetime import datetime

# Retrieve the current date and time
date = datetime.today().strftime("%Y-%m-%d")
timezone = datetime.now().astimezone().tzinfo

In [12]:
query = f"""
Book slots according to the folloing todo list:
    1PM - 3PM -> Code,
    5PM - 7PM -> Meeting,
    9AM - 12AM -> Learn something,
    8PM - 10PM -> Game
Label them with the work provided to be done in that time period. Schedule it for today. Today's date is {date} (it's in YYYY-MM-DD format) and make the timezone be {timezone}.",
"""

In [13]:
for chunk in app.stream({"messages": [
    ("system", SYSTEM_PROMPT),
    ("human", query)
]}, stream_mode="values"):
        chunk["messages"][-1].pretty_print()

================================ Human Message =================================


Book slots according to the folloing todo list:
    1PM - 3PM -> Code,
    5PM - 7PM -> Meeting,
    9AM - 12AM -> Learn something,
    8PM - 10PM -> Game
Label them with the work provided to be done in that time period. Schedule it for today. Today's date is 2024-11-28 (it's in YYYY-MM-DD format) and make the timezone be +07.",



[2024-11-28 02:55:19,134][INFO] Executing `GOOGLECALENDAR_CREATE_EVENT` with params={'summary': 'Code', 'start_datetime': '2024-11-28T13:00:00+07:00', 'event_duration': '2h'} and metadata={} connected_account_id=None
[2024-11-28 02:55:19,148][INFO] Executing `GOOGLECALENDAR_CREATE_EVENT` with params={'summary': 'Game', 'start_datetime': '2024-11-28T20:00:00+07:00', 'event_duration': '2h'} and metadata={} connected_account_id=None
[2024-11-28 02:55:19,168][INFO] Executing `GOOGLECALENDAR_CREATE_EVENT` with params={'summary': 'Learn something', 'start_datetime': '2024-11-28T09:00:00+07:00', 'event_duration': '3h'} and metadata={} connected_account_id=None
[2024-11-28 02:55:19,183][INFO] Executing `GOOGLECALENDAR_CREATE_EVENT` with params={'summary': 'Meeting', 'start_datetime': '2024-11-28T17:00:00+07:00', 'event_duration': '2h'} and metadata={} connected_account_id=None


================================== Ai Message ==================================
Tool Calls:
  GOOGLECALENDAR_CREATE_EVENT (e1d380ef-fac7-4a0e-81e3-9bcdf9d79e29)
 Call ID: e1d380ef-fac7-4a0e-81e3-9bcdf9d79e29
  Args:
    event_duration: 2h
    start_datetime: 2024-11-28T13:00:00+07:00
    summary: Code
  GOOGLECALENDAR_CREATE_EVENT (8bb5b101-660b-4dbd-afb6-a9b889f41a30)
 Call ID: 8bb5b101-660b-4dbd-afb6-a9b889f41a30
  Args:
    event_duration: 2h
    summary: Meeting
    start_datetime: 2024-11-28T17:00:00+07:00
  GOOGLECALENDAR_CREATE_EVENT (fb7e2413-e9ee-435c-a9a9-11c69a245ddd)
 Call ID: fb7e2413-e9ee-435c-a9a9-11c69a245ddd
  Args:
    event_duration: 3h
    summary: Learn something
    start_datetime: 2024-11-28T09:00:00+07:00
  GOOGLECALENDAR_CREATE_EVENT (19cd9e8e-1013-425f-aac7-3ae3dc93cafd)
 Call ID: 19cd9e8e-1013-425f-aac7-3ae3dc93cafd
  Args:
    event_duration: 2h
    summary: Game
    start_datetime: 2024-11-28T20:00:00+07:00


[2024-11-28 02:55:22,323][INFO] Got response={'successfull': True, 'data': {'response_data': {'kind': 'calendar#event', 'etag': '"3465474643802000"', 'id': 's42mb6gsheqadl72omputmrle8', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=czQybWI2Z3NoZXFhZGw3Mm9t...
[2024-11-28 02:55:22,528][INFO] Got response={'successfull': True, 'data': {'response_data': {'kind': 'calendar#event', 'etag': '"3465474644314000"', 'id': 'hpjlv7kahlfe741gpdrao5nct8', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=aHBqbHY3a2FobGZlNzQxZ3Bk...
[2024-11-28 02:55:22,740][INFO] Got response={'successfull': True, 'data': {'response_data': {'kind': 'calendar#event', 'etag': '"3465474644742000"', 'id': 'hn44c863oas73kigj45qc5gups', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=aG40NGM4NjNvYXM3M2tpZ2o0...
[2024-11-28 02:56:22,340][INFO] Got response={'successfull': True, 'data': {'response_data': {'kind': 'calendar#event', 'eta

================================= Tool Message =================================
Name: GOOGLECALENDAR_CREATE_EVENT

{"successfull": true, "data": {"response_data": {"kind": "calendar#event", "etag": "\"3465474643802000\"", "id": "s42mb6gsheqadl72omputmrle8", "status": "confirmed", "htmlLink": "https://www.google.com/calendar/event?eid=czQybWI2Z3NoZXFhZGw3Mm9tcHV0bXJsZTggY29tYmk5MTQ0QG0", "created": "2024-11-27T19:55:21.000Z", "updated": "2024-11-27T19:55:21.901Z", "summary": "Game", "creator": {"email": "combi9144@gmail.com", "self": true}, "organizer": {"email": "combi9144@gmail.com", "self": true}, "start": {"dateTime": "2024-11-28T20:00:00+07:00", "timeZone": "Asia/Ho_Chi_Minh"}, "end": {"dateTime": "2024-11-28T22:00:00+07:00", "timeZone": "Asia/Ho_Chi_Minh"}, "iCalUID": "s42mb6gsheqadl72omputmrle8@google.com", "sequence": 0, "reminders": {"useDefault": true}, "eventType": "default"}}, "error": null}
================================== Ai Message ==================================

O